In [ ]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
from plotnine import *
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score #model evaluation

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


%matplotlib inline

In [ ]:
# Data
m = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Music_data.csv")

m.head()

In [ ]:
# shape

# Train Test Split

In [ ]:
# Split Data



In [ ]:
# size of train and test

In [ ]:
# create and fit

In [ ]:
# predictions


In [ ]:
#testing R2

In [ ]:
#training R2

In [ ]:
# true vs. pred

In [ ]:
# plot

# K Fold

In [ ]:
# kf

In [ ]:
# set up

In [ ]:
# loop

    

In [ ]:
# mse

In [ ]:
# mean mse

In [ ]:
# r2

In [ ]:
# mean r2

# Leave One Out

In [ ]:
# loo set up


In [ ]:
# loo loop

In [ ]:
# mean mse

# Always Get the Same Split!
When you do KFold or TTS, you may get slightly different splits every time you run the function, even with the same data! This is because the data is split randomly. 

If you want the same split every time, use the `random_state` argument.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(m[predictors],
                                                m["danceability"],
                                                test_size=0.2,
                                                random_state = 42)
kf = KFold(n_splits = 10, shuffle = True, random_state = 42) #we'll talk more about this later...

# Quick Note about Data Leakage...

In [ ]:
# Say you want to standardize...
# Split Data
predictors = ["energy", "key", "loudness", "mode", "speechiness", "acousticness",
              "instrumentalness", "liveness", "valence", "duration_ms"]
contin = ["energy", "loudness", "speechiness", "acousticness",
              "instrumentalness", "liveness", "valence", "duration_ms"]


# tts

In [ ]:
# z

In [ ]:
# in KF
